In [1]:
NAME = 0
OPERATOR = 1
PUSH = 2
POP = 3
ATTR = 4
NUMBER = 5
EOF = 6
EQUAL = 7
NOTEQUAL = 8
LESSTHAN = 9
GREATERTHAN = 10
LESSEQUAL = 11
GREATEREQUAL = 12
DOT = 13
E_SYNTAX = 14
PLUS = 15
MINUS = 16
DIV = 17
MULT = 18
NUMBER = 19
UNARY_PLUS = 20
UNARY_MINUS = 21
POW = 22
CALL_FUNCTION = 23
COMMA = 24
NTYPES = 25


NUM2STR = [
    "NAME",
    "OPERATOR", 
    "PUSH", 
    "POP", 
    "ATTR", 
    "NUMBER", 
    "EOF", 
    "==", 
    "!=", 
    "<", 
    ">", 
    "<=", 
    ">=", 
    ".", 
    "E_SYNTAX", 
    "+", 
    "-", 
    "/", 
    "*",
    "NUMBER",
    "+",
    "-",
    "**",
    "CALL_FUNCTION",
    "COMMA",
]

class Token():
    def __init__(self, typ, val):
        self.type = typ
        self.val = val
    def __repr__(self):
        return "<%s:%s>"%(NUM2STR[self.type], self.val)

class TokenState():
    def __init__(self, s):
        self.i = -1
        self.n = -1
        self.s = s
        self.l = len(s)
        self.tokens = []
        
    def nextc(self):
        self.i += 1
        if self.i >= self.l:
            self.i = self.l
            return '\0'
        return self.s[self.i]
    
    def peekc(self, c):
        return self.peek() == c
            
    def peek(self):
        i = self.i+1
        if i >= self.l:
            return '\0'
        return self.s[i]
    
    def ungetc(self):
        self.i -= 1
        
    def endtok(self):
        return self.s[self.n:self.i+1]
    
    def starttok(self):
        self.n = self.i

def getc(s):
    return s.nextc()

def ungetc(s):
    return s.ungetc()
    
def maybe_name(c):
    o = ord(c)
    if 97 <= o <= 122: return True
    if 65 <= o <= 90 : return True
    if c == "_": return True
    return False

def isnumberstart(c):
    return c in "0123456789-."

def isnumber(c):
    return c in "0123456789."

def parseop(s,c):
    s.starttok()
    c2 = getc(s)
    if c == "=":
        if c2 != "=":
            ungetc(s)
        val = EQUAL
    elif c == "<":
        if c2 == ">":
            val = NOTEQUAL
        elif c2 == "=":
            val = LESSEQUAL
        else:
            ungetc(s)
            val = LESSTHAN
    elif c == ">":
        if c2 == "=":
            val = GREATEREQUAL
        else:
            ungetc(s)
            val = GREATERTHAN
    elif c == ".":
        ungetc(s)
        val = DOT
    elif c == "!":
        if c2 != "=":
            op = E_SYNTAX
            val = E_SYNTAX
    elif c == "+":
        ungetc(s)
        val = PLUS
    elif c == "-":
        ungetc(s)
        val = MINUS
    elif c == "/":
        ungetc(s)
        val = DIV
    elif c == "*":
        if c2 == "*":
            val = POW
        else:
            ungetc(s)
            val = MULT
    return Token(OPERATOR, val)
        
def is_name(c):
    return maybe_name(c) or (48 <= ord(c) <= 57) # include numbers

def tokenize(st):
    s = TokenState(st)
    stack = []
    def token(t,v=None): stack.append(Token(t,v))
    def lasttype(): return stack[-1].type if stack else NTYPES
    
    level = 0
    while True:
        c = getc(s)
        if c == '\0':
            break  # EOF
        elif c == " ":
            continue
        elif maybe_name(c):               # NAME
            s.starttok()
            while is_name(c):
                c = getc(s)
            ungetc(s)
            name = s.endtok()
            
            if c == '(':
                token(CALL_FUNCTION, name)
                getc(s)  # discard
                level += 1
            else:
                token(NAME, name)
        elif isnumber(c) or (c == "-" and isnumber(s.peek()) and 
                        (lasttype() == OPERATOR or lasttype() == NTYPES)):  # distinguish operator from unary minus
            s.starttok()
            c = getc(s)         # numbers that start with "-" fail isnumber() without this
            while isnumber(c):
                c = getc(s)
            ungetc(s)
            token(NUMBER, (s.endtok()))
        elif c in "({[":                  # PUSH
            token(PUSH, c)
            level += 1
        elif c in ")}]":                  # POP
            token(POP, c)
            level -= 1
        elif c in "<>=+-/*.":
            tok = parseop(s, c)
            if tok.type == E_SYNTAX:
                raise SyntaxError("Error at position %s"%s.i)
            # handle "5+-4" etc
            if lasttype() == OPERATOR:
                if tok.val == PLUS:
                    tok.val = UNARY_PLUS
                elif tok.val == MINUS:
                    tok.val = UNARY_MINUS
                else:
                    raise SyntaxError("Consecutive token mismatch: %r -> %r"% \
                                    NUM2STR(stack[-1].val), tok.val)
            stack.append(tok)
        elif c == ",":
            token(OPERATOR, COMMA)
        else:
            raise SyntaxError(repr(c))
    if level != 0:
        raise SyntaxError("Mismatched brackets")
    return stack
            
            

In [2]:
def bin_op_func(f):
    def func(a,b):
        rv = f(a.val, b.val)
        return Token(NUMBER, rv)
    return func

import operator
OPERATORS = {}
OPERATORS[PLUS] = bin_op_func(operator.add)
OPERATORS[MINUS] = bin_op_func(operator.sub)
OPERATORS[MULT] = bin_op_func(operator.mul)
OPERATORS[DIV] = bin_op_func(operator.truediv)
OPERATORS[POW] = bin_op_func(operator.pow)

In [76]:

def ftest(a=Token(NAME, "def a"),b=Token(NAME, "def b")):
#     print("a:", a.val)
#     print("b:", b.val)
    rv = a.val+b.val
    if isinstance(rv, str):
        return Token(NAME, rv)
    elif isinstance(rv, (float, int)):
        return Token(NUMBER, rv)
    else:
        raise TypeError(type(rv))
    
NAMES = {}
NAMES["ftest"] = ftest

def qeval2(l):
    if isinstance(l, str):
        l = tokenize(l)
    ops = []
    stack = []
    tok_iter = iter(l)
    next_tok = lambda: next(tok_iter, None)
    while True:
        tok = next_tok()
        if tok is None:
            break
        handle_tok(ops, stack, tok_iter, tok)
    stack_finish(stack, ops)
    mstack = exec_stack(stack)
    return mstack.pop().val
            
            
def handle_tok(ops, stack, tok_iter, tok):
    if tok.type == NAME:
        stack.append(tok)
    elif tok.type == NUMBER:
        tok.val = float(tok.val)
        stack.append(tok)
    elif tok.type == CALL_FUNCTION:
        eval_function(ops, stack, tok_iter, tok)
    elif tok.type == OPERATOR:
        eval_operator(ops, stack, tok_iter, tok)
    elif tok.type == PUSH:
        eval_push(ops, stack, tok_iter, tok)
    elif tok.type == POP:
        eval_pop(ops, stack, tok_iter, tok)
    else:
        raise ValueError("NotImplemented: %r"%tok)
        
PREC = {}
PREC[PLUS] = 0
PREC[MINUS] = 0
PREC[MULT] = 1
PREC[DIV] = 1
PREC[DOT] = 2
PREC[POW] = 3
PREC[CALL_FUNCTION] = 10

LEFT = 0
RIGHT = 1

ASC = {}
ASC[PLUS] = LEFT
ASC[MINUS] = LEFT
ASC[MULT] = LEFT
ASC[DIV] = LEFT
ASC[DOT] = 2
ASC[POW] = RIGHT
ASC[CALL_FUNCTION] = RIGHT


def eval_operator(ops, stack, tok_iter, tok):
    o = tok.val
    if o == COMMA:
        while ops:
            stack.append(ops.pop())
        return
    while ops:
        v = ops[-1].val
        if v == "(":
            break
        if PREC[o] > PREC[v] or (PREC[o] == PREC[v] and ASC[o] == "RIGHT"):
            break
        stack.append(ops.pop())
    ops.append(tok)
    
def eval_pop(ops, stack, tok_iter, tok):
        while ops[-1].type != PUSH:
            stack.append(ops.pop())
        ops.pop()
        
def eval_push(ops, stack, tok_iter, tok):
    ops.append(tok)
    
def stack_finish(stack, ops):
    while ops:
        stack.append(ops.pop())
    
def eval_function(ops, stack, tok_iter, tok):
    func = NAMES[tok.val]
    # set the val after saving the name so that 
    # the CALL_FUNCTION token works properly with
    # eval_operator to consume pending operations
    # (e.g. func(1+1)) properly before execution.
    tok.val = CALL_FUNCTION
    ops.append(tok)
    fstack = []
    fops = []
    level = 0
    while True:
        tok = next(tok_iter)
        if tok.type == POP:
            if level == 0:
                #import pdb; pdb.set_trace()
                stack_finish(fstack, fops)
                mstack = exec_stack(fstack)
                rv = func(*mstack)
                stack.append(rv)
                return
            else:
                level -= 1
        elif tok.type == PUSH:
            level += 1
        handle_tok(fops, fstack, tok_iter, tok)
        
def exec_stack(stack):
    mstack = []
    for v in stack:
        if v.type == NUMBER or v.type==NAME:
            mstack.append(v)
        elif v.type == OPERATOR:
            func = OPERATORS[v.val]
            a = mstack.pop()
            b = mstack.pop()
            val = func(b, a)
            mstack.append(val)
    return mstack

In [77]:
s=list(tokenize("ftest(1+2*3, 4)"))
s

[<CALL_FUNCTION:ftest>,
 <NUMBER:1>,
 <OPERATOR:15>,
 <NUMBER:2>,
 <OPERATOR:18>,
 <NUMBER:3>,
 <OPERATOR:24>,
 <NUMBER:4>,
 <POP:)>]

In [78]:
qeval2(s)

11.0

In [64]:
import random

def rand_float(n=10):
    return "%.*f"%(random.randint(0,3), random.random()*n)
    
def gen_expr():
    exp = []
    lvl = 0
    n = 0
    k = 0
    if random.randint(1,20) < 15:
        return rand_float()
    while True:
        if not k:
            v = rand_float()
            exp.append(v)
        k = 0
        op = random.choice(('*', '/', '+', '-'))
        exp.append(op)
        if n > 0:
            n -= 1
        parenth = random.randint(1, 20) < 5
        if parenth:
            exp.append('(')
            n = 1
            lvl += 1
        if n <= 0:
            if lvl == 0 and random.randint(1, 20) < 5:
                break
            elif lvl > 0:
                close = random.randint(1, 20) < 12
                if close:
                    exp.append(rand_float())
                    exp.append(")")
                    lvl -= 1
                    k = 1
                    
    exp.append(rand_float())
    return " ".join(map(str, exp))

In [68]:
def test(n=10):
    for i in range(n):
        s1 = gen_expr()
        s2 = gen_expr()
        st = "ftest(%s, %s)" % (s1, s2)
        try:
            rq = qeval2(st)
        except ZeroDivisionError:
            continue
        try:
            r1 = eval(s1)
        except ZeroDivisionError:
            continue
        try:
            r2 = eval(s2)
        except ZeroDivisionError:
            continue
        exp = r1 + r2
        b = rq == exp
        if not b:
            print(b, st, rq, r1+r2)
#     if b:
#         print(b, st)
#     else:
#         print(b, st, rq, r1 + r2)

In [69]:
qeval2("ftest(1-0,3)")

4.0

In [79]:
test(1000)

In [80]:
qeval2("1+ftest(1,1)")

3.0

In [81]:
qeval2("1+ftest(2,3)")

6.0